## Analyse the spin enumeration results

Is it necessary to enumerate the spin?

In [1]:
%aiida

Loaded AiiDA DB environment - profile name: scanlon-test.

In [2]:
from toolchest.hull import get_enumerate_records

In [3]:
sgroup_run2 = Group.objects.get(label='li-fe-o-s/run2-airss-refine-structures')
sgroup_run1 = Group.objects.get(label='li-fe-o-s/run1-airss-refine-structures')

In [4]:
enum_df = get_enumerate_records([sgroup_run1, sgroup_run2], 520)

Entries for magnetic enumeration calculations: 518


  0%|          | 0/518 [00:00<?, ?it/s]

In [11]:
enum_df['n_fe'] = enum_df['pmg_struct'].apply(lambda x: x.composition['Fe'])
enum_df['chemsys'] = enum_df['pmg_struct'].apply(lambda x: x.composition.chemical_system)
enum_df['natoms'] = enum_df['structure'].apply(lambda x: len(x.sites))
enum_df['e_pa'] = enum_df['energy'] / enum_df['natoms']

In [7]:
from aiida_user_addons.vworkflows.magnetic import SpinEnumerateWorkChain

In [8]:
from aiida.orm import StructureData

In [9]:
q = QueryBuilder()
q.append(Group, filters={'id': {'in': [sgroup_run1.id, sgroup_run2.id]}})
q.append(Node, with_group=Group)
q.append(WorkflowFactory('vaspu.relax'))
q.append(StructureData, project=['label', '*'])
q.append(SpinEnumerateWorkChain, project=['uuid'])
q.append(Dict, project=['attributes'])
lable, structure, uuid, data = q.first()

In [11]:
selected = ['Li2FeSO', 'Li2Fe2S2O', 'Li4Fe3S3O2', 'Li2Fe4S3O2']
def print_de(label, structure, data):
    fme = data['energy'][0]
    emin = min(data['energy'])
    de = emin - fme
    ps = structure.get_pymatgen()
    ps.add_oxidation_state_by_guess()
    comp_reduced = structure.get_pymatgen().composition.get_reduced_composition_and_factor()[0]
    if comp_reduced.reduced_formula not in selected:
        return
    nfe = comp_reduced['Fe']
    natom = comp_reduced.num_atoms
    if ps.composition['Fe2+'] > 0:
        print(f'Fe2+: {comp_reduced.reduced_formula:<15}: {de*1e3 / nfe:.3f} ({de * 1e3 / natom:.3f}) meV {label}')
    else:
        print(f'Fe3+: {comp_reduced.reduced_formula:<15}: {de*1e3 / nef:.3f} ({de * 1e3 / natom:.3f}) meV {label}')
    
uuids = []
for label, structure, uuid, data in q.all():
    print_de(label, structure, data)
    uuids.append(uuid)

Fe2+: Li4Fe3S3O2     : -68.568 (-17.142) meV LFSO-Li4Fe3S3O2-200926-154112-68ece2 RELAXED
Fe2+: Li2FeSO        : -67.409 (-13.482) meV LFSO-Li2FeSO-200912-001832-2b9b69.res
Fe2+: Li2Fe4S3O2     : -63.587 (-23.123) meV LFSO-Li2Fe4S3O2-200912-051915-e358d4.res
Fe2+: Li2Fe2S2O      : -46.806 (-13.373) meV LFSO-Li2Fe2S2O-200913-112906-1643ea.res
Fe2+: Li2FeSO        : -62.710 (-12.542) meV LFSO-Li2FeSO-201025-015107-3451c2 RELAXED
Fe2+: Li2Fe4S3O2     : -54.790 (-19.923) meV LFSO-Li2Fe4S3O2-201025-153957-9403b6 RELAXED
Fe2+: Li2Fe2S2O      : -31.977 (-9.136) meV LFSO-Li2Fe2S2O-201022-042623-e6d3a9 RELAXED
